In [1]:
import os
import shutil
import json
import glob
# import docx
# import torch
# import numpy as np
# import pandas as pd
import openai
from openai import OpenAI
from tqdm.notebook import tqdm
# from docx import Document
# from docxcompose.composer import Composer
# from docx.enum.text import WD_ALIGN_PARAGRAPH
# from docx.shared import Pt
# import markdown
# import markdown2
# import pdfkit
# from pygments.formatters import HtmlFormatter
# import pypandoc
from bs4 import BeautifulSoup
import re
# from weasyprint import HTML

In [2]:
# # Path to the JSON file where you want to save the mapping
# json_file_path = "/Users/alaa/Documents/ucsf/data/addo_dc/gpt_data/filename_mapping.json"

# # Writing the dictionary to the JSON file
# with open(json_file_path, 'r') as json_file:
#     transcript_id_map = json.load(json_file)
# print(transcript_id_map)

In [3]:
# def add_transcript_id_to_docs(file_id_map: dict, file_dir: str):
#     for original_filename, filename in transcript_id_map.items():
#         file_id = filename.split('_')[1].split('.')[0]
#         filepath = f"{file_dir}/{filename}"
#         # Open the document
#         doc = Document(filepath)
#         # Add the Transcript ID at the beginning
#         doc_paragraph = doc.paragraphs[0]
#         doc_paragraph.insert_paragraph_before(f"Transcript ID: {file_id}")
#         # Save the document with the updated content
#         doc.save(filepath)
# # add_transcript_id_to_docs(file_id_map=transcript_id_map, 
# #                           file_dir="/Users/alaa/Documents/ucsf/data/addo_dc/gpt_data")

In [4]:
all_contents = []

In [5]:
# List all application file and structure
all_application_entities = glob.glob("/home/justcallmeal/matrix/mason/mason_backend/**/*", recursive=True)
# Filter to only include files (exclude directories)
all_application_filepaths = [f for f in all_application_entities if not os.path.isdir(f) \
                             and 'README' not in f.split('/')[-1] \
                             and 'pyc' not in f.split('/')[-1] \
                             and 'table' not in f.split('/')[-1] \
                             and 'column' not in f.split('/')[-1] \
                             and 'extra' not in f.split('/')[-1] \
                            and 'zip' not in f.split('/')[-1]]
print(len(all_application_filepaths))

68


In [6]:
# List all application file and structure
all_fe_entities = glob.glob("/Users/alaa/Documents/swiftcraft/code/swiftcraft_frontend/src/**/*.js", recursive=True)
# Filter to only include files (exclude directories)
all_fe_filepaths = [f for f in all_fe_entities if not os.path.isdir(f) \
                             and 'README' not in f.split('/')[-1] \
                             and 'pyc' not in f.split('/')[-1] \
                             and 'table' not in f.split('/')[-1] \
                             and 'column' not in f.split('/')[-1] \
                             and 'extra' not in f.split('/')[-1] \
                            and 'zip' not in f.split('/')[-1]]
print(len(all_fe_entities))

0


In [7]:
agent_task = """You will assist in the development of a FastAPI application for SwiftCraft.
Here we provide each of the files that belong to the codebase. 
You are always expected to work off of the existing codebase, 
modifying appropriate existing files or creating new files only if needed."""

In [8]:
# "/Users/alaa/Documents/swiftcraft/code/swiftcraft_backend/email_service_account/swiftcraftweb-21142c8c9e06.json"

In [9]:
# Define the keywords
keywords = ["custom-timeouts", "schemas", "models", "database",
            # ".nginx",
            # "allow.conf",
            # "env_vars",
            "main.py",
            # "account",
            # "stakeholder",
            # "agent_tech", "agent_mech", 
            # "time_utils.py",
            # "email_utils.py",
            "email.py",
            # "email_service.py",
            # "gmail",
            # "refresh_gmail_watch",
            # "check_active_projects",
            # "email_service.py",
            # "communication",
            # "communication_service",
            # "sms_service",
            "sms",
            # "simulation",
            # "stakeholder",
            # "file_service",
            # "simulation",
            # "agent",
            # "agent_service",
            # "instruction",
            # "instruction_service",
            "activation", 
            # "account",
            # "file",
            # "activation_service.py",
            "project.py", 
            "project_service.py",
            # "check_active_projects.py",
#             "instruction"
           ]

all_messages = [{"role": "user", "content": agent_task}]

for fp in all_application_filepaths:
    # Only proceed if the file path contains at least one keyword
    if any(keyword in fp for keyword in keywords):
        if 'outlook_email.py' in fp or 'outlook_email_service.py' in fp:
            continue
        if 'gmail_email.py' in fp or 'gmail_email_service.py' in fp:
            continue
        if 'gmail_auth.py' in fp or 'gmail_auth_service.py' in fp:
            continue
        filename = fp.split('/')[-1]
        print(filename)
        with open(fp, 'r') as file:
            file_content = file.read()
        message_content = f"file name: {filename}\n{file_content}"
        message = {"role": "user", "content": message_content}
        all_messages.append(message)

print(len(all_messages))

database.py
main.py
models.py
schemas.py
project_service.py
activation_service.py
sms_service.py
sms.py
project.py
email.py
activation.py
12


In [10]:
# all_messages[6]

In [11]:
# query_id = 28000

In [12]:
# # all_messages = [{"role": "user", "content": agent_task}]
# for fp in all_fe_entities:
#     filename = fp.split('/')[-1]
#     print(filename)
#     try:
#         with open(fp, 'r') as file:
#             file_content = file.read()
#         message_content = f"file name: {filename}\n{file_content}"
#         message = {"role": "user", "content": message_content}
#         all_messages.append(message)
#     except:
#         continue

In [16]:
# query_id = 1`0000

In [17]:
query_id += 10

user_query = """I need to implement a new feature Project Agent Override. For this, we need to allow the following flow from the UI/UX to be possible:

- Open the Activations Page as an account type of admin
- Click on an active project, then click a Project Agent Override button, which then allows the user to view the project pages (Overview, Communications, Contacts, Files)
- Throughout all pages in the project, there needs to be a clear flag indicating that this is in override mode
- If the user (admin account holder) creates any communication inside the project, it must come from the Project Agent as the sender.

Provide suggested changes to the API codebase to facilitate this feature. Focus more on an elegant and effective design pattern; less on the exact code changes.


"""

user_message = {"role": "user", "content": user_query}


In [18]:
# with open(f'/home/justcallmeal/matrix/mason/docs/agent_output_test90020_raw.md', 'r', encoding='utf-8') as f:
#     raw_output = f.readlines()
# raw_output = ''.join(raw_output)

In [27]:
query_id += 1
agent_message = {"role": "user", "content": raw_output}
user_query2 = """Okay looks good. Now try to implement it by suggesting specific changes to the codebase."""
user_reply = {"role": "user", "content": user_query2}

In [28]:
# query_id += 1
# err_msg = """
# """
# err_message = {"role": "user", "content": err_msg}

In [29]:
agent_input = all_messages + [user_message] + [agent_message] + [user_reply]
#+ [err_message]
agent_input

[{'role': 'user',
  'content': 'You will assist in the development of a FastAPI application for SwiftCraft.\nHere we provide each of the files that belong to the codebase. \nYou are always expected to work off of the existing codebase, \nmodifying appropriate existing files or creating new files only if needed.'},
 {'role': 'user',
  'content': 'file name: database.py\nfrom sqlalchemy import create_engine\nfrom sqlalchemy.ext.declarative import declarative_base\nfrom sqlalchemy.orm import sessionmaker\nimport os\n\n# Remove load_dotenv() since we\'ll set environment variables in Elastic Beanstalk\n# from dotenv import load_dotenv\n# load_dotenv()\n\nDATABASE_URL = (\n    f"postgresql://{os.getenv(\'DB_USER\')}:"\n    f"{os.getenv(\'DB_PASSWORD\')}@"\n    f"{os.getenv(\'DB_HOST\')}:{os.getenv(\'DB_PORT\')}/"\n    f"{os.getenv(\'DB_NAME\')}"\n)\n\nengine = create_engine(DATABASE_URL)\nSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)\nBase = declarative_base()\n

In [30]:
client = OpenAI()
response = client.chat.completions.create(
    model="o3",
    reasoning_effort="high",
    messages=agent_input
)
print(response.choices[0].message.content)

Below is a “concrete-enough” patch-list that can be dropped into the existing tree and compiled / unit-tested right away.  
(All snippets are self-contained; if a file is not mentioned you do **not** have to touch it.)

─────────────────────────────────────────────────────────────────────────────
1.  app/core/request_context.py   (NEW)
─────────────────────────────────────────────────────────────────────────────
```python
# app/core/request_context.py
from dataclasses import dataclass
from typing import Optional
from app import models            # ← already in sys.path because of “app.” package


@dataclass
class RequestContext:
    """
    ‘Envelope’ that accompanies every authenticated request.

    • account               – the real caller (Admin, PM, normal user …)
    • is_override           – True  ⇢  we are looking at a Project Agent override
    • project_id            – project that is currently impersonated
    • agent_stakeholder_id  – stakeholder record that represents the 

In [31]:
raw_output = response.choices[0].message.content

In [32]:
with open(f'/home/justcallmeal/matrix/mason/docs/agent_output_test{query_id}_raw.md', 'w', encoding='utf-8') as f:
    f.write(raw_output)

In [33]:
print(f'/docs/agent_output_test{query_id}_raw.md')

/docs/agent_output_test110011_raw.md
